# Compute climate index TG ('Mean daily mean temperature'). The computation in the back is taken from the python module xclim

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyhomogenize import open_xrdataset

import index_calculator as xcalc

Select one CORDEX `tas` file and open it with xarray.

In [3]:
ifile = "tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_20010101-20051231.nc"

In [4]:
ds = open_xrdataset(ifile)
ds

<xarray.Dataset>
Dimensions:                     (time: 1826, bnds: 2, rlat: 412, rlon: 424,
                                 vertices: 4)
Coordinates:
  * time                        (time) object 2001-01-01 12:00:00 ... 2005-12...
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    height                      float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds                   (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    tas                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
Attributes: (12/34)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    title:                          GERICS-REMO2015 model output prepared for...
    parent_experiment:              N/A
    modeling_realm:                 atmos
    realization:                    3
    cmor_version:                   2.9.1
    tracking_id:                    hdl:21.14103/55a844fd-666d-4799-8563-8fbd...

Now do a preprocessing of the dataset and create an index_calculator.PreProcessing object.
Preprocessor tasks are:
* select a user-given time range (parameter: time_range)
* check time axis for missing, redundant and duplicated time steps
* crop time axis to required left and right border
* write available and requested time range to object

In [5]:
preproc = xcalc.preprocessing(ds)
preproc.preproc

/home/ludwig/miniconda3/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/home/ludwig/miniconda3/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]


<xarray.Dataset>
Dimensions:                     (time: 1826, bnds: 2, rlat: 412, rlon: 424,
                                 vertices: 4)
Coordinates:
  * time                        (time) object 2001-01-01 12:00:00 ... 2005-12...
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    height                      float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds                   (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    tas                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
Attributes: (12/34)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    title:                          GERICS-REMO2015 model output prepared for...
    parent_experiment:              N/A
    modeling_realm:                 atmos
    realization:                    3
    cmor_version:                   2.9.1
    tracking_id:                    hdl:21.14103/55a844fd-666d-4799-8563-8fbd...

Now, use the above object to compute the climate index `TG`.

In [6]:
proc = xcalc.processing("TG", preproc_obj=preproc)
proc.proc

/home/ludwig/miniconda3/lib/python3.8/site-packages/xarray/coding/times.py:360: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]


<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 5,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) object 2001-07-03 00:00:00 ... 2005-07...
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) object 2001-01-01 12:00:00 ......
Attributes: (12/34)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    title:                          GERICS-REMO2015 model output prepared for...
    parent_experiment:              N/A
    modeling_realm:                 atmos
    realization:                    3
    cmor_version:                   2.9.1
    tracking_id:                    hdl:21.14103/55a844fd-666d-4799-8563-8fbd...

Finally, do some postprocessing. The postprocessor writes some additional attributes to `ds`' global attributes.

In [7]:
postproc = xcalc.postprocessing(
    project="CORDEX",
    proc_obj=proc,
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    institution_id="GERICS",
)
postproc.postproc

/home/ludwig/git/index_calculator/index_calculator/_utils.py:48: UserWarning: No contact is selected. 'contact=...'
  warnings.warn(msg)
/home/ludwig/git/index_calculator/index_calculator/_ci_netcdfattrs.py:103: UserWarning: Could not set attribute ci_reference_period
  warnings.warn("Could not set attribute {}".format(attribute))


<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 5,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) object 2001-07-03 00:00:00 ... 2005-07...
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) object 2001-01-01 12:00:00 ......
Attributes: (12/44)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    ci_name:                        TG
    ci_frequency:                   year
    ci_timerange_index:             2001-2005
    ci_timerange_source:            20010101-20051231
    ci_package_name:                xclim_0.37.0
    ci_package_reference:           xcalc_0.3.2

Optionaly, we can store the dataset as a netCDF file on disk. The output file naming depends on the user-given project. 

In [8]:
xcalc.outputwriter(postproc_obj=postproc)

KeyboardInterrupt: 

For simplyfication, we can do preprocessing, processing, postprocessing and output file storing by calling just one function.

In [ ]:
idx_obj = xcalc.index_calculator(
    ds=ds,
    index="TG",
    project="CORDEX",
    institution_id="GERICS",
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    write=True,
)

In [ ]:
idx_obj.postproc